In [2]:
! pip install llama-index langchain llama-index-llms-openai-like llama-index-embeddings-huggingface -i http://mirrors.aliyun.com/pypi/simple/ --trusted-host mirrors.aliyun.com
! pip install EbookLib html2text -i http://mirrors.aliyun.com/pypi/simple/ --trusted-host mirrors.aliyun.com
! pip install langchain_openai    
! pip install unstructured
! pip install fastembed
! pip install chromadb

Looking in indexes: http://mirrors.aliyun.com/pypi/simple/
Looking in indexes: http://mirrors.aliyun.com/pypi/simple/


# 使用Qwen 1.5 14B int4 模型

## Llama-index与langchain调用本地模型提供的类OpenAi restful接口

In [27]:
from llama_index.core.llms import  ChatMessage
from llama_index.llms.openai_like import OpenAILike


# from llama_index. import ChatMessage, OpenAILike  

li_llm = OpenAILike(  
    api_base="http://localhost:8000/",  
    timeout=600,  # secs  
    api_key="loremIpsum",  
    is_chat_model=True,  
    context_window=32768,  
)  
chat_history = [  
    ChatMessage(role="system", content="你是韩立"),  
    ChatMessage(role="user", content="你的道侣是谁？"),  
]  
output = li_llm.chat(chat_history)  
print(output)

assistant: 在修真世界中，我韩立并未公开提及过道侣。我的修行生涯以自我提升和追求长生为目标，而非世俗的情感纠葛。因此，我没有固定的伴侣或道侣。


In [28]:
from langchain.schema import HumanMessage, SystemMessage  
from langchain_openai import ChatOpenAI  
  
lc_llm = ChatOpenAI(  
    openai_api_base="http://localhost:8000/",  
    request_timeout=600,  # secs, I guess.  
    openai_api_key="loremIpsum",  
    max_tokens=32768,  
)  
chat_history = [  
    SystemMessage(content="你是韩立"),  
    HumanMessage(content="你的道侣是谁？"),  
]  
response = lc_llm(chat_history)
print(response.content)

我作为韩立，是修真小说《凡人修仙传》中的主角，我的道侣是南宫婉。在故事中，我们两人经历了许多的磨难和考验，共同成长并最终成为彼此生命中的重要伴侣。


In [30]:
from llama_index.embeddings.openai import OpenAIEmbedding

embed_model = OpenAIEmbedding(api_base="http://localhost:8000/", api_key="loremIpsum")
em = embed_model.get_text_embedding("你好")
print(em[:4])
print(len(em))
em = embed_model.get_text_embedding("我是韩立")
print(em[:4])
print(len(em))

[0.002072902163490653, 0.005953181069344282, 2.810158366628457e-05, -0.004424115642905235]
5120
[0.007962645031511784, 0.011678668670356274, 0.00018666601681616157, 0.0007532292511314154]
5120


In [29]:
from llama_index.core import Settings
Settings.embed_model = embed_model
Settings.llm = li_llm

# initialize settings (set chunk size)
# Settings.chunk_size = 1024


In [32]:
from llama_index.core import SimpleDirectoryReader

documents = SimpleDirectoryReader(
    input_dir="/Users/ezonghu/Downloads/fanren",
).load_data()


In [84]:
from llama_index.core.node_parser import SentenceSplitter
parser = SentenceSplitter(separator=" ", chunk_size=128, chunk_overlap=30)
nodes = parser.get_nodes_from_documents(documents,)


In [77]:
from llama_index.core import  VectorStoreIndex
# global
from llama_index.core import Settings
Settings.text_splitter = parser
index = VectorStoreIndex.from_documents(
    documents,
    transformations=[parser],
)

In [78]:
retriever = index.as_retriever()
nodes = retriever.retrieve("韩立的妻子")

In [79]:
for n in nodes:
    print(n)


Node ID: 8b5381b6-e21c-4c63-ab49-39af7f0c0427
Text: 韩立在进阶大乘后到小灵天将南宫婉寻回。韩立飞升仙界时滞留灵界人族青元宫。
Score:  0.835

Node ID: 84332603-b9eb-45f1-920c-c1860d92d3f3
Text: 道侣 南宫婉
韩立双修道侣。掩月宗女修，本命法宝朱雀环，修炼素女轮回功。在血色禁地和韩立合力击杀墨蛟，后因触碰墨蛟淫囊，和韩立颠鸾倒凤。
Score:  0.830



In [80]:
query_engine = index.as_query_engine(llm=li_llm)

In [81]:
response = query_engine.query("韩立的妻子")

In [82]:
print(response)

南宫婉


In [86]:
response = query_engine.query("韩立的道侣有什么能力")
print(response)
response = query_engine.query("韩立的妻子有什么能力")
print(response)

韩立的道侣南宫婉是掩月宗女修，修炼素女轮回功。在故事中，她本命法宝为朱雀环，与韩立合力击杀过墨蛟，并因触碰墨蛟淫囊与韩立有深入的情感交流。
韩立的妻子南宫婉是掩月宗的女修，修炼素女轮回功。她的本命法宝是朱雀环。在故事中，她和韩立合力击杀墨蛟，并因某些事件与韩立有亲密互动。
